# Credit Card Retention Analysis
Dataset Description

    A manager at the bank is disturbed with more and more customers leaving their credit card services. They would really like to understand what characteristics lend themselves to someone who is going to churn so they can proactively go to the customer to provide them better services and turn customers' decisions in the opposite direction.

    This dataset consists of 10,000 customers mentioning their age, salary, marital_status, credit card limit, credit card category, etc. There are nearly 18 features.

    16.07% of customers have churned.

In order to read in a csv into python, we will be leveraging the Pandas library. Any package we want to use in Python will need an import statement. In addition to pandas which we will import using import pandas as pd, we will also import matplotlib and seaborn (libraries used for visualization) and numpy (a library for array manipulation).
Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot
sns.set()
pd.options.display.max_columns = 999

# Reading in Dataset

The method we will be using is pd.read_csv which implies we will be reading a comma separated value file. You can see this in the defaults for this method by typing help(pd.read_csv) and see that the separator is set to , with other helpful defaults like header='infer'. You can read through the rest to get familiar with parameters you can pass through that might be specific to what you may need and different from the defaults.

If you type pd.read and then press tab you will see other methods available to you out of the box to read in files. Examples: pd.read_excel, pd.read_pickle, pd.read_json


In [ ]:
data = pd.read_csv('BankChurners_v2.csv')

The next steps I always do after reading in my file is to:

    1. data.shape to see the size of the dataset. The size will help me decide on how to manage working with the dataset if it happens to be large. Here we see this dataset has 10K+ rows of customer data and 23 columns describing the behavior of those customers.

    2. data.head() to see the top of the dataset and make any changes like renaming column names. The default will show the top 5 rows, but you can pass through any number you like (10,25, etc)

    3. data.columns to see what all the column names

Let's do that here.


In [ ]:
data.shape

In [ ]:
data.head()

# General Cleaning Techniques

The next important step when working with any dataset, is to perform any necessary cleaning steps. This includes (but is not limited to): 1) converting incorrect variable data types, 2) dropping or imputing missing (NULL) values, 3) finding and fixing erroneous values, and 4) handling outliers.
Checking for Duplicates

A really good high level check to do at the start is to check for duplicates in the dataset. If there is a unique index you can check on like customer or advertiser IDs then that's the best variable to check uniques on by using the nunique() method. Else, you can use the .drop_duplicates() method.


In [ ]:
data['CLIENTNUM'].nunique()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

# Subsetting Data

Next, we will look at the column names and see if we want to change column names or subset the data.

In [ ]:
data.columns

In [ ]:
data = data[['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',]]

# Datatypes

Next, we will take a look at datatypes. To check for datatypes, we will type df.dtypes to see how each variable has been read in. Main things to check here are dates and numbers that have been read in as string values and will need to be converted into their respective types in order to work with those variables as intended. Here we see that we have three different datatypes int64, float64 and object. The object dtype is roughly analogous to str in native Python. You can reference the user guide for pandas dtypes.


In [ ]:
data.dtypes

# Missing Values

Next we will check for NULL or missing values in the dataset. We can check this with a simple one-liner. By running the below code, we see that Education Level has 1519 instances of missing vlaues, Marital Status has 749 and Income Category has 1112. Since these are categorical variables, we can replace the null values with Unknown. However, if we were dealing with numerical missing values, we would likely impute (or fill in) the missing records with a mean or median.

A more in-depth review of how to handle different types of missing values is here.

Let's replace these missing records!


In [ ]:
data.isnull().sum()

In [ ]:
data[data['Education_Level'].isna()]

To fill in the missing values, we will use the following code:

In [ ]:
data['Education_Level'] = data['Education_Level'].fillna('Unknown')
data['Marital_Status'] = data['Marital_Status'].fillna('Unknown')
data['Income_Category'] = data['Income_Category'].fillna('Unknown')

We can (and should) check that the code we ran did what we expected, and we can verify like this:

In [ ]:
data.isnull().sum()

# Data Transformation: Binning

Another part of the data cleaning process can include creating any new variables that we may need later. Here, having each customer's age is helpful, but we could also look at them as groups: people in their 20s, 30s, etc. Let's take a look at how we can create these categories.

To bin our ages, we will need a couple data points: the min and max ages. Let's find those.


In [ ]:
print(data['Customer_Age'].min())
print(data['Customer_Age'].max())

To create bins, we can use the df.cut() method. Parameters we will need to pass in are the bin intervals (which are set to be inclusive by default, but we can override using right=False) and labels for those bins. Thus, we will use the following code to define our age brackets:

In [ ]:
# https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-5-binning-c5bd5fd1b950
bins = [25, 30, 40, 50, 60, 70, 80]
labels = ['20s', '30s', '40s', '50s', '60s', '70s']
data['Customer_Age_bins'] = pd.cut(data['Customer_Age'], bins=bins, labels=labels, include_lowest=True, right=False)

Let's verify that the behavior we expected from this method resulted in the right output. Since we overrode the inclusive values, we should see that 30 should show in the 30s bin vs the 20s bin.

In [ ]:
data[data['Customer_Age']==30]

# EDA (exploratory vs explanatory)

We will now be moving into what I consider to be the funnest part of any analysis. This is where we will get to explore the data in any which way we want in order to find the "story" within the data. Here we know we care about attrition and attributes that are leading to attrition in the hopes of advising the company on how to reduce lost customers.

In Cole Knaflic's Storytelling with Data, she covers the difference between exploratory and explanatory analysis and that will be something we emphasize here as well. Typical pitfalls I see when building a final deliverable is to include what was meant to be exploratory analysis. Basically I define this as visuals for ME-- and visuals for YOUR AUDIENCE. Visuals for me can be messy, complex, and anything that helps lead me to the next step or to a conclusion. From there, I can develop a visual for YOUR AUDIENCE that simplifies the finding in a way you can understand in 10 seconds or less.

# Sanity Checks

Let's start by confirming to ourselves the composition of our client data. A part of any good analysis is continued sanity checks, so let's verify that 16% of our dataset are attrited customers. We can do this simply by looking at counts in each bucket and leveraging the .value_counts() method in python. This method will count the number of instances in the dataset that fall into either category.


In [ ]:
data['Attrition_Flag'].value_counts()

In [ ]:
data['Attrition_Flag'].value_counts()['Attrited Customer'] / data.shape[0]

In [ ]:
print(round(data['Attrition_Flag'].value_counts()['Attrited Customer'] / data.shape[0] * 100 , 2) , '% of our customers have churned , which matches the documentation')

# Summary Statistics

To start to understand our dataset a bit more, we will want to produce summary or descriptive statistics that can tell us more about the variables and features within the dataset. Typically, we are looking to understand:

1) how many instances are in the dataset (frequency or counts) 
2) a measure of central tendency (mean, median, mode)
3) the spread of the dataset (variance, standard deviation)

Depending on what variable or feature you are looking at, a different measure of central tendency may help represent that data better. It's important to understand the differences between them and when to use which so you can make the right choice.

The Mean is the average of all values in a dataset, while the Median represents the midpoint of the values (50% above and 50% below. For example, you have a list y consisting of numbers 0-100 in intervals of 10 as below:


In [ ]:
y = list(range(0, 110, 10))
print(y)

In [ ]:
mean = sum(y)/len(y) # the way to get the count of a list is by using the len() method
print(mean)

In [ ]:
np.mean(y)

In [ ]:
np.median(y)

If I add one more data point to the set, 900, let's see how things change:

In [ ]:
y.append(900)

In [ ]:
np.mean(y)

In [ ]:
np.median(y)

Notice how the Mean jumped up, highly impacted by that new data point (outlier), while the median came up slightly, but was much less sensitive. In the case where there are outliers, the Median is a much better representation of central tendency than the Mean.

In python, we can use the .describe() method to see these metrics for all the numerical variables in the dataset including: quantiles, min, max and std.

std helps us understand how spread out the values of that variable are ---> the bigger the std the bigger the spread


In [ ]:
data.describe()

Here we can see things like:

1) The longest customer in this dataset has been around for 56 months or about 4 years and a half. (Max)
2) The average number of relationships a customer has is ~4. (Mean and median agree here)
3) The average credit limit is $8.6K, but the median credit limit is much lower at $4.5K. (signals some skew in this variable) 



In [ ]:
print('The average Total_Relationship_Count is', round(np.mean(data['Total_Relationship_Count']), 2), 'and the median is', np.median(data['Total_Relationship_Count']))

In [ ]:
print('The average Credit_Limit is $', round(np.mean(data['Credit_Limit']), 2), 'and the median is $', np.median(data['Credit_Limit']))

# Distributions

Let's now take a look at our three measures of central tendency visually to see how they are impacted by skews and outliers. As you can see, the Mode will always represent the highest point or the point of highest density on the distribution plot. The Median will always split the dataset in half (50/50). And the Mean will lean towards the direction of the skew.

The most common way to evaluate distributions is through Histograms which look a lot like bar charts, but have very important differences. Histograms specifically look to visualize frequency distributions where the height of the bin will tell you approx how many observations lie within that bin/bucket/range.

Let's start with Customer Age.

In [ ]:
plt.hist(data['Customer_Age']);

matplotlob has many parameters you can use to customized this chart, but the most important one for histograms are bins. Bins are set to 10 by default and are a way to group the values together to allow a high level distribution to render. However, this is a good one to adjust if the bins are too coarse and are not allowing us to fully see the distribution.

Based on our diagrams above, Customer_Age looks fairly normally distributed. Nice, this tells us that it's possible to find normally distributed data in the wild, which is good! To read this chart, the yaxis represents frequencies, or in this case Customers and the xaxis represents the age bins. So, approximately ~2400 customers are within 45-50. This is not meant to be an exact measure, but more an indication of the distribution. To get exact numbers, we will want to dig into the data, but this is our first step to guide us where to look.

Let's take a quick look at the distribution for Months_on_book.


In [ ]:
plt.hist(data['Months_on_book']);

It looks pretty normal outside of a very strong peak at ~36 months (roughly 3500 customers). If we had access to the owner to ask more questions about this, we would want to know if this was a data error (default months for those without a record), or if there was a major marketing campaign that brought in a lot of users 36 months ago.

Now let's take a look at the Credit_Limit variable that we looked at before:


In [ ]:
plt.hist(data['Credit_Limit']);

In [ ]:
plt.hist(data['Credit_Limit']);
plt.vlines(data['Credit_Limit'].mean(), 0, 5000)
plt.vlines(data['Credit_Limit'].median(), 0, 5000) 

In [ ]:
data['Credit_Limit'].median() # can't see the median line here...let's fix!

Let's improve this visual a bit by increasing the bin size, changing the color of the lines and adding some annotations. Next, let's add a title and some x and y labels as well.

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(data['Credit_Limit'], bins=30);

plt.vlines(data['Credit_Limit'].mean(), 0, 2500, colors='Black')
plt.vlines(data['Credit_Limit'].median(), 0, 2500, colors='Black')
plt.text(data['Credit_Limit'].mean()-1000, 2500+50, "Mean")
plt.text(data['Credit_Limit'].median()-1000, 2500+50, "Median")
plt.ylim(0, 2800);
plt.title("Histogram of Customer Credit Limit");
plt.ylabel('Frequency');
plt.xlabel('Credit Limit');

MUCH better. Now, it's clear that this Credit Limit is a skewed variable, being skewed higher (or to the right) by a few customers with high Credit Limits.

Based on its relation to our feet, we know this is right skewed.

Let's try Total_Trans_Ct that looked to have a somewhat similar mean and median.


In [ ]:
plt.hist(data['Total_Trans_Ct']);

It might not be obvious that this is a bimodal distribution, let's increase the bin size. Default is 10. Bimodal data typically will show two peaks of data-- which usually indicates you’ve got two different groups.

In [ ]:
plt.hist(data['Total_Trans_Ct'], bins=50);

# Data Transformations: Normalization and Log

While we are still on the topic of distributions, it's important to note that there are simple ways to transform a skewed distribution into one that looks more "normal" or to normalize between two values. Let's take a look at a few together.

We will cover two very common transformations:

1) Normalization (also referred to as a min-max scaler): Normalization will convert all data points to values between two values (usually 0 and 1). 
2) Log transformation: This helps make the data "less skewed". When using log with Python, the default base is usually e.

We will use the out of the box log transformation from the numpy library and write our own function for normalization.

In [ ]:
def normalize(column):
    upper = column.max()
    lower = column.min()
    y = (column - lower)/(upper-lower)
    return y

In [ ]:
data['Credit_Limit_Normalized'] = normalize(data['Credit_Limit'])
data['Credit_Limit_Log_Transformed'] = np.log(data['Credit_Limit'])

To see exactly how the transformations impact the variables we are looking at, we will use seaborn subplots to plot them side by side using subplots.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15,10))
fig.suptitle('Before and After Transformation')

#create boxplot in each subplot
sns.histplot(data, x="Credit_Limit", ax=axes[0,0])
sns.histplot(data, x="Credit_Limit_Normalized", ax=axes[0,1])
sns.histplot(data, x="Credit_Limit", ax=axes[1,0])
sns.histplot(data, x="Credit_Limit_Log_Transformed", ax=axes[1,1]);

As you can see above, when we Normalized the values, they maintained their shape but now are bound between 0 and 1. This method is helpful when comparing across many variables of different magnitudes (ex. Number of Relationships which ranges between 1-6 vs Credit Limit which ranges between 0 and 35K).

However, it's clear that the Log-Transformation made an impact on the distribution!


# Other Distributions Plots
Box and Wisker Plot

Other ways to look at distributions include Box and Whisker plots, Pyramid Chart, Candlestick charts, etc. We will review Box and Whisker and Pyramid Charts together. Let's look at Total Transaction Count by Gender on a Box and Whisker Plot.

The box and whisker plot allows the analyst to quickly find and identify the median, quartiles and any outliers.


In [ ]:
sns.boxplot(x=data["Gender"], y=data['Total_Trans_Ct'])

This plot quickly shows us that the median transaction account is slightly higher for females than males and outliers seem to be present among both groups.

# Pyramid Chart

What if the question we wanted to know is if there was a difference in the distribution of customers by age and gender? A Pyramid chart could get us there!

For this, we will need to prepare our data. Remember the Customer_Age_bins we made before? We will use them now.

To get the data in the format we need, we will need to aggregate the data up to the Age and Gender level. I will do that now with this line of code and we will go over aggregations and groupbys later. For now, let's take a look at the output together.


In [ ]:
pyramid = data.groupby(['Gender', 'Customer_Age_bins'])['CLIENTNUM'].nunique().reset_index()

In [ ]:
pyramid

With a few code line swaps to read in our dataset, we can create the following. This is a small reminder to leverage what exists on the internet. A lot of work will be done for you, and the job will be to know how to find it and translate it for your use-case!

In [ ]:
women_bins = np.array(-1* pyramid[pyramid['Gender']=='F']['CLIENTNUM'])
men_bins = np.array(pyramid[pyramid['Gender']=='M']['CLIENTNUM'])

y = list(range(20, 100, 10))

layout = go.Layout(yaxis=go.layout.YAxis(title='Age'),
                   xaxis=go.layout.XAxis(
                       range=[-3000, 3000],
                       tickvals=[-2500,-2000,-1500,-1000,-500, 0, 500,1000,1500,2000,2500],
                       ticktext=[2500,2000,1500,1000,500, 0, 500,1000,1500,2000,2500],
                       title='Customers'),
                   barmode='overlay',
                   bargap=0.1)

p_data = [go.Bar(y=y,
               x=men_bins,
               orientation='h',
               name='Men',
#                text=men_bins.astype('int'),
               hoverinfo='x',
               marker=dict(color='powderblue')
               ),
        go.Bar(y=y,
               x=women_bins,
               orientation='h',
               name='Women',
               text=-1 * women_bins.astype('int'),
               hoverinfo='text',
               marker=dict(color='seagreen')
               )]

iplot(dict(data=p_data, layout=layout))

Here we see there is no real difference in the distribution of Customers by Age and Gender. The majority of customers are in their 40s--a similar finding to the one earlier when we were looking at histograms.

# Comparing Categories

Comparing histograms to bar charts, let's take a look at the same dimensions we used above (Age and Gender) and use a barplot to visualize these groups. The main difference here will be that we are not looking at frequencies anymore, we are specifying and plotting a measure of our choice. Here, we will look at Credit_Limit by Age and Gender. Let's plot!


In [ ]:
sns.barplot(x='Customer_Age_bins', y='Credit_Limit', data=data, estimator=np.mean);

Notice how I've passed through an estimator. When the data is given at the Customer level (not aggregated), then it will estimate the values for that group (the default is the mean, but we can switch that out). You'll notice error bars and this is to show that the bar height ends where the mean for that group is, but there are Customers with more or less, as dictated by the length of the error bar.

Let's see what happens when we add in Gender, which we can do by specifying the Hue.


In [ ]:
sns.barplot(x='Customer_Age_bins', y='Credit_Limit', hue='Gender', data=data, estimator=np.mean);

Notice how Females are getting credit limits much lower than Males! Interesting insight. Just to sanity check ourselves, let's try the median instead of the mean here. Let's swap that using the estimator parameter.

In [ ]:
sns.barplot(x='Customer_Age_bins', y='Credit_Limit', hue='Gender', data=data, estimator=np.median);

Looks like it's true that men on average are getting accepted for much more than women are for our company. If we wanted a bar chart without the error bars, we would need to do a quick aggregation. Let's run the next line of code together and go over it in more detail in the next section.

In [ ]:
barplot = data.groupby(['Customer_Age_bins', 'Gender'])['Credit_Limit'].mean().reset_index()
barplot

In [ ]:
sns.barplot(x='Customer_Age_bins', y='Credit_Limit', hue='Gender', data=barplot);

We can turn the visual so that the bars go horizontally with just one parameter change. Let's try! Here we swap the default orient to "h".

In [ ]:
sns.barplot(x='Credit_Limit', y='Customer_Age_bins', hue='Gender', data=barplot, orient='h');

There are times where a bar chart may have too many categories that are too close to tell the difference between the heights. A small adaptation from the Bar Chart is the Lollipop Chart, let's make it together.

In [ ]:
lollipop = data.groupby(['Customer_Age_bins'])['Credit_Limit'].mean().reset_index().sort_values('Credit_Limit')
lollipop

In [ ]:
plt.stem(lollipop['Customer_Age_bins'], lollipop['Credit_Limit'], linefmt='--');

Most common, is to see this chart with the bars flowing horizontally and sorted from highest to lowest so we can visually pick up how the categories compare with one another. Otherwise they can be more confusing to read so use your best judgement on when to use them. We can do that here with the following code:

In [ ]:
# https://jonathansoma.com/lede/foundations-2018/pandas/creating-lollipop-charts-with-pandas-and-matplotlib/
fig, ax = plt.subplots()
ax.hlines(lollipop['Customer_Age_bins'], xmin=0, xmax=lollipop['Credit_Limit'])
ax.plot(lollipop['Credit_Limit'], lollipop['Customer_Age_bins'], "o", color='black');

# Data Visualization: Data Tables

Aggregations and Groupby: If you are familiar with Pivot Tables in excel, than the way that .groupby() works with pandas dataframes should be pretty intuative! What both of these functions will try to accomplish is aggregations and ways of summarizing the data. They are both flexible enough to allow you to pull in different aggregation types (sum, mean, counts, etc). To read more about the difference between pivot tables in python and groupby, see this guide.

Coming back to our business problem, we want to understand something about Customers who are leaving. Let's see if we can see anything obvious about their behavior:


In [ ]:
data.groupby([ 'Attrition_Flag']).agg({
               'CLIENTNUM':'nunique',
               'Customer_Age': 'mean', 
               'Dependent_count': 'mean',
               'Months_on_book': 'mean',
               'Total_Relationship_Count': 'mean',
               'Months_Inactive_12_mon': 'mean',
               'Contacts_Count_12_mon': 'mean',
               'Credit_Limit': 'mean',
               'Total_Revolving_Bal': 'mean',
               'Avg_Open_To_Buy': 'mean',
               'Total_Amt_Chng_Q4_Q1': 'mean',
               'Total_Trans_Amt': 'mean',
               'Total_Trans_Ct': 'mean',
               'Total_Ct_Chng_Q4_Q1': 'mean',
               'Avg_Utilization_Ratio': 'mean'
            })

To see the groups more clearly, we can use the Transpose function. This will swap Columns and Rows.

In [ ]:
data_pivot = data.groupby(['Attrition_Flag']).agg({
                           'CLIENTNUM':'nunique', # number of unique customers in each group
                           'Customer_Age': 'mean', # the rest are looking at the mean per group
                           'Dependent_count': 'mean',
                           'Months_on_book': 'mean',
                           'Total_Relationship_Count': 'mean',
                           'Months_Inactive_12_mon': 'mean',
                           'Contacts_Count_12_mon': 'mean',
                           'Credit_Limit': 'mean',
                           'Total_Revolving_Bal': 'mean',
                           'Avg_Open_To_Buy': 'mean',
                           'Total_Amt_Chng_Q4_Q1': 'mean',
                           'Total_Trans_Amt': 'mean',
                           'Total_Trans_Ct': 'mean',
                           'Total_Ct_Chng_Q4_Q1': 'mean',
                           'Avg_Utilization_Ratio': 'mean'
                        })

data_pivot.T

# Relationships

The next thing we may be interested in is how variables move with (positively correlated) or against (negatively correlated) each other. We can do this quickly for the whole numerical dataset (or a subset) using a pairplot or pairgrid.

This is a nice visualization since it will show relationships between each numerical variable and every other one in the dataset as well as histograms along the diagonal.


In [ ]:
# https://stackoverflow.com/questions/25039626/how-do-i-find-numeric-columns-in-pandas
numeric_data = data._get_numeric_data()

In [ ]:
numeric_data.columns

Since this is a computationally expensive operation, and we will not gain new information from some of these variables (like CLLIENTNUM) we will simplify even further.

In [ ]:
numeric_data = data[['Credit_Limit', 'Total_Revolving_Bal', 'Months_on_book',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Attrition_Flag']]

In [ ]:
g = sns.PairGrid(numeric_data, diag_sharey=False, corner=True)
g.map_lower(sns.scatterplot)
g.map_diag(sns.histplot)

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(x='Total_Trans_Amt', y='Total_Trans_Ct', data=data);

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(x='Total_Trans_Amt', y='Total_Trans_Ct', data=data, hue='Attrition_Flag')

Looks like the top group doesn't have a single attrited customer! And that number looks to be around $11K.

Finding #1: No attrited customer above $11K of spend
